## Unzip files

In [42]:
# !unzip '/content/drive-download-20240813T132337Z-001.zip'

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import transforms
from timm import create_model
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import mean_squared_error, r2_score

# Tentukan perangkat yang akan digunakan (GPU jika tersedia, atau CPU jika tidak)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Dataset Class
class CephalometricDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        image_name = self.landmarks_frame.iloc[idx, 1]
        image_path = f'{self.image_folder}/{image_name}'
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (256, 256))
        landmarks = self.landmarks_frame.iloc[idx, 4:].values.astype('float32').reshape(-1, 2)

        if self.transform:
            image = self.transform(image)

        # Tambahkan dimensi channel baru untuk kompatibilitas dengan HRNet
        image = image.unsqueeze(0)
        return image, landmarks

# 2. Data Loading and Preprocessing
transform = transforms.Compose([  # Transformasi khusus untuk data pelatihan
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.ColorJitter(brightness=(0.5, 1.5), contrast=(0.5, 1.5), saturation=0.5, hue=0.5),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-5, 5))], p=0.5),
])

csv_file = 'data_annot.csv'
image_folder = 'data/images/'
dataset = CephalometricDataset(csv_file, image_folder, transform=transform)

# Set seed untuk reprodusibilitas
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Membagi dataset menjadi train, validation, dan test sets
dataset_size = len(dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)

# Tentukan ukuran split
train_split = int(0.8 * dataset_size)
val_split = int(0.1 * dataset_size)
test_split = dataset_size - train_split - val_split

# Dapatkan indeks untuk setiap set
train_indices = indices[:train_split]
val_indices = indices[train_split:train_split + val_split]
test_indices = indices[train_split + val_split:]

# Buat objek Subset untuk setiap set
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Buat DataLoader untuk setiap set
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model_path = 'best_model_mre_0.001_adamax.pth'
asli = pd.read_csv('data_annot.csv')

def mean_euclidean_distance(x_coords_true, y_coords_true, x_coords_pred, y_coords_pred):
    # Menghitung rata-rata Euclidean distance
    distances = np.sqrt((x_coords_true - x_coords_pred) ** 2 + (y_coords_true - y_coords_pred) ** 2)
    return np.mean(distances)

def evaluate_model(test_loader, model_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    hrnet = create_model('hrnet_w18', pretrained=False)
    hrnet.classifier = nn.Linear(hrnet.classifier.in_features, 32 * 2)

    state_dict = torch.load(model_path, map_location=device)
    state_dict.pop('conv1.weight', None)
    state_dict.pop('conv1.bias', None)
    hrnet.load_state_dict(state_dict, strict=False)

    first_conv_layer = hrnet.conv1
    new_first_conv_layer = nn.Conv2d(1, first_conv_layer.out_channels,
                                     kernel_size=first_conv_layer.kernel_size,
                                     stride=first_conv_layer.stride,
                                     padding=first_conv_layer.padding,
                                     bias=first_conv_layer.bias is not None)
    hrnet.conv1 = new_first_conv_layer

    hrnet.to(device)
    hrnet.eval()

    med_results = []
    with torch.no_grad():
        for images, landmarks in test_loader:
            images = images.to(device)
            landmarks = landmarks.to(device)
            images = images.squeeze(1)  # Pastikan hanya satu channel, sesuai dengan modifikasi di conv1
            outputs = hrnet(images)

            # Konversi output ke numpy dan reshape sesuai dengan jumlah landmarks
            outputs_np = outputs.cpu().numpy().reshape(-1, 32, 2)
            landmarks_np = landmarks.cpu().numpy().reshape(-1, 32, 2)

            # Hitung MED per gambar
            for output, landmark in zip(outputs_np, landmarks_np):
                x_coords_pred = output[:, 0]
                y_coords_pred = output[:, 1]
                x_coords_true = landmark[:, 0]
                y_coords_true = landmark[:, 1]

                # Hitung Euclidean distance per landmark
                med = mean_euclidean_distance(x_coords_true, y_coords_true, x_coords_pred, y_coords_pred)
                med_results.append(med)

    # Return rata-rata dari semua MED
    return pd.DataFrame({'MED': med_results})

import logging
import pandas as pd
import numpy as np
from scipy.spatial import distance

# Enable logging to check intermediate values
logging.basicConfig(level=logging.INFO)

# Ambang batas MED untuk menganggap nilai sebagai outlier
MED_THRESHOLD = 100

def prediksi_dan_visualisasikan(image_path, model_path):
    # Load model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    hrnet = create_model('hrnet_w18', pretrained=False)
    hrnet.classifier = nn.Linear(hrnet.classifier.in_features, 32 * 2)

    # Load state dict without conv1 layer if it exists
    state_dict = torch.load(model_path, map_location=device)
    if 'conv1.weight' in state_dict:
        state_dict.pop('conv1.weight')
    if 'conv1.bias' in state_dict:
        state_dict.pop('conv1.bias')
    hrnet.load_state_dict(state_dict, strict=False)

    # Create new first conv layer to accept a single-channel grayscale image
    first_conv_layer = hrnet.conv1
    new_first_conv_layer = nn.Conv2d(1, first_conv_layer.out_channels,
                                     kernel_size=first_conv_layer.kernel_size,
                                     stride=first_conv_layer.stride,
                                     padding=first_conv_layer.padding,
                                     bias=first_conv_layer.bias is not None)
    hrnet.conv1 = new_first_conv_layer

    hrnet.to(device)
    hrnet.eval()

    # Load original image to get its size
    original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    original_height, original_width = original_image.shape[:2]
    logging.info(f"Original Image Size: {original_width}x{original_height}")

    # Resize image for model input
    resized_image = cv2.resize(original_image, (256, 256))
    image = transform(resized_image).unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        outputs = hrnet(image)
        predicted_landmarks = outputs.view(-1, 32, 2).cpu().numpy()

    # Rescale the predicted landmarks back to the original image size
    x_scale = original_width / 256
    y_scale = original_height / 256
    predicted_landmarks[:, 0] = predicted_landmarks[:, 0] * x_scale
    predicted_landmarks[:, 1] = predicted_landmarks[:, 1] * y_scale

    logging.info(f"Predicted Landmarks (Scaled): {predicted_landmarks}")

    return predicted_landmarks

def match_and_calculate_med(x_coords_true, y_coords_true, x_coords_pred, y_coords_pred):
    # Use Hungarian Algorithm or nearest neighbor for matching points
    true_points = np.column_stack((x_coords_true, y_coords_true))
    pred_points = np.column_stack((x_coords_pred, y_coords_pred))

    if len(true_points) != len(pred_points):
        logging.warning("Jumlah titik prediksi dan ground truth tidak sama. Menggunakan nearest neighbor untuk matching.")
        distances = distance.cdist(true_points, pred_points, 'euclidean')
        row_ind, col_ind = distance.matching(distances)
        matched_pred_points = pred_points[col_ind]
    else:
        matched_pred_points = pred_points

    x_coords_pred_matched = matched_pred_points[:, 0]
    y_coords_pred_matched = matched_pred_points[:, 1]

    # Hitung Euclidean Distance
    med_values = np.sqrt((x_coords_true - x_coords_pred_matched)**2 + (y_coords_true - y_coords_pred_matched)**2)

    med = np.mean(med_values)

    return x_coords_pred_matched, y_coords_pred_matched, med

# Mulai modifikasi kode untuk melakukan looping chosen_index dari 0 sampai 31
# Inisialisasi list untuk menyimpan hasil
x_coords_true_list = []
y_coords_true_list = []
x_coords_pred_list = []
y_coords_pred_list = []
med_list = []
euclidean_distance_list = []

# Loop untuk memproses tiap indeks dari 0 sampai 31
for chosen_index in range(len(test_dataset)):
    sample_index = test_dataset.indices[chosen_index]

    file_name = asli.iloc[sample_index]['file_name']
    image_path = f'data/images/{file_name}'

    # Dapatkan prediksi landmarks
    predicted_landmarks = prediksi_dan_visualisasikan(image_path, model_path)

    # Jika bentuknya (1, 32, 2), perlu direshape
    if predicted_landmarks.shape[0] == 1:
        predicted_landmarks = predicted_landmarks.reshape(32, 2)

    x_coords_pred = predicted_landmarks[:, 0]
    y_coords_pred = predicted_landmarks[:, 1]

    # Filter asli untuk file_name yang acak
    asli_1 = asli[asli['file_name'] == file_name]
    landmark_columns = asli_1.columns[asli_1.columns.str.endswith(('_x', '_y'))]
    x_coords_true = asli_1[landmark_columns[landmark_columns.str.endswith('_x')]].values.flatten()
    y_coords_true = asli_1[landmark_columns[landmark_columns.str.endswith('_y')]].values.flatten()

    # Hitung MED dengan pasangan titik yang paling dekat dan ambil nilai yang dicocokkan
    x_coords_pred_matched, y_coords_pred_matched, med = match_and_calculate_med(x_coords_true, y_coords_true, x_coords_pred, y_coords_pred)

    # Simpan hasil
    x_coords_true_list.append(x_coords_true)
    y_coords_true_list.append(y_coords_true)
    x_coords_pred_list.append(x_coords_pred_matched)
    y_coords_pred_list.append(y_coords_pred_matched)
    med_list.append(med)
    euclidean_distance_list.append(np.sqrt((x_coords_true - x_coords_pred_matched)**2 + (y_coords_true - y_coords_pred_matched)**2))

# Menghitung rata-rata dan standar deviasi untuk tiap kolom
mean_x_true = np.mean(x_coords_true_list, axis=0)
mean_y_true = np.mean(y_coords_true_list, axis=0)
mean_x_pred = np.mean(x_coords_pred_list, axis=0)
mean_y_pred = np.mean(y_coords_pred_list, axis=0)
mean_med_per_landmark = np.mean(euclidean_distance_list, axis=0)
std_dev_med_per_landmark = np.std(euclidean_distance_list, axis=0)

# Membuat DataFrame akhir untuk visualisasi dan analisis
final_data = {
    'X Ground Truth Mean': mean_x_true,
    'Y Ground Truth Mean': mean_y_true,
    'X Predicted Mean': mean_x_pred,
    'Y Predicted Mean': mean_y_pred,
    'Mean Euclidean Distance': mean_med_per_landmark,
    'Standard Deviation MED': std_dev_med_per_landmark
}
final_df = pd.DataFrame(final_data)
final_df['Mean ± SD'] = final_df.apply(lambda row: f"{row['Mean Euclidean Distance']:.2f} ± {row['Standard Deviation MED']:.2f}", axis=1)

# Tampilkan DataFrame akhir
final_df


,X Ground Truth Mean,Y Ground Truth Mean,X Predicted Mean,Y Predicted Mean,Mean Euclidean Distance,Standard Deviation MED,Mean ± SD
0,365.4,573.1,767.656372,1251.042236,790.302291,96.913663,790.30 ± 96.91
1,325.8,593.3,974.976440,1729.021851,1309.526203,120.540875,1309.53 ± 120.54
2,371.3,410.4,370.301880,421.739441,21.245921,21.062257,21.25 ± 21.06
3,232.8,405.7,250.821625,411.461975,27.082433,32.780581,27.08 ± 32.78
4,361.6,253.7,388.388489,261.343933,49.962610,73.562685,49.96 ± 73.56
5,170.0,516.5,159.764603,500.932312,45.398552,63.521724,45.40 ± 63.52
6,375.8,463.2,376.611023,448.016296,35.684728,48.365824,35.68 ± 48.37
7,367.6,573.2,367.086975,558.784241,21.721677,13.748661,21.72 ± 13.75
8,360.9,601.3,361.858246,609.939331,42.282203,55.454078,42.28 ± 55.45
9,187.7,288.6,194.923691,277.560455,36.616920,39.761908,36.62 ± 39.76
